In [4]:
import feniax_sensitivity
import bug_sensitivity_thickness
import bug_handler
import pch_parser

import matplotlib.pyplot as plt
from pyNastran.op2.op2 import read_op2
from importlib import reload

In [5]:
reload(feniax_sensitivity)
reload(bug_sensitivity_thickness)
reload(bug_handler)
reload(pch_parser)
from pch_parser import *
from bug_handler import *
from feniax_sensitivity import *
from bug_sensitivity_thickness import *

In [6]:
NASTRAN_LOC='cmd.exe /c C:/MSC.Software/MSC_Nastran/20182/bin/nast20182.exe'

In [7]:
fname='../NASTRAN/BUG_103cap.bdf'
model=NastranDesignModel_thickness(fname)

In [8]:
ymax=model.bdf_handler.node_coords[:,1].max()
xmax=model.bdf_handler.node_coords[:,0].max()
coord_control_y=np.linspace(0,ymax,4)
coord_control1=np.stack([np.zeros_like(coord_control_y),coord_control_y],axis=1)
coord_control2=np.stack([xmax*np.ones_like(coord_control_y),coord_control_y],axis=1)
coord_control=np.concatenate([coord_control1,coord_control2],axis=0)
param=np.array([0.1,0.4,0.2,0.1])
param=np.concatenate([param,param],axis=0)

In [9]:
dir_input='./input/'
dir_output='./output/'
if not os.path.exists(dir_input):
  os.makedirs(dir_input)
if not os.path.exists(dir_output):
  os.makedirs(dir_output)
nastranhandler=NastranHandler(model,dir_input,dir_output)
sensitivitynastran=SensitivityNastran(model,dir_input,dir_output)

In [10]:
min_thick_ratio=0.5
max_thick_ratio=1.5

args=(coord_control,min_thick_ratio,max_thick_ratio)

In [11]:
Ka,Ma=nastranhandler.get_rom(param,*args)

In [12]:
sensitivitynastran=SensitivityNastran(model,dir_input,dir_output,delta=1e-2)
dKa,dMa=sensitivitynastran.get_rom_sensitivity(param,*args)

In [13]:
fname='../NASTRAN/BUG_103cao.bdf'
model=NastranDesignModel_thickness(fname)
sensitivitynastran=SensitivityNastran(model,dir_input,dir_output,delta=1e-2)
dvec,dval=sensitivitynastran.get_eig_sensitivity(param,*args)